<h1>Python + Selenium </h1>

* Instalar o Selenium
#abrir o prompt e dar pip install selenium
* Baixar o WebDriver (chromedriver)
<br>#webdriver é um arquivo que vai permitir o python a integrar ao navegador; para isso é necessário pegar o webdriver dos navegadores distintos que vc irá usar.
<br>#colocar o arquivo onde está instalado o python(python.exe), no caso em C:\Users\User\anaconda3\

In [ ]:
from selenium import webdriver
navegador = webdriver.Chrome()
##navegador.get vai até uma pagina
site_investimento = ("https://fiis.com.br/")
#print(site_investimento)
navegador.get(site_investimento)
##procura pelo xpath e clica
#navegador.find_element("xpath", '//*[@id="fii-section"]/div/div/div[2]/div[3]/div/div[1]/div[2]/div/span').click()

In [ ]:
import pandas as pd
#Criaremos um dataframe coms os ativos que vão ser pesquisados
ativos = pd.DataFrame({"nome":["AFHI11","SDIL11","XPML11","IRDM11","RBVA11","RBED11","FLMA11","TRXF11","HGRU11","TRBL11","KNSC11","MALL11"]})
ativos

,nome
0,AFHI11
1,SDIL11
2,XPML11
3,IRDM11
4,RBVA11
5,RBED11
6,FLMA11
7,TRXF11
8,HGRU11
9,TRBL11


In [ ]:
from selenium.webdriver.common.by import By
#Importaremos a biblioteca By
from io import StringIO     # Usamos StringIO para simular um arquivo em memória


#Faremos uma iteração aqui, criando o indicador IND, procurando pelo index de ativos(ativos.index)
for ind in ativos.index:
    pesquisa = ativos.loc[ativos.index[ind]]    #pesquisa vai guardar as infos de cada index, começando do 0
    #print(pesquisa)                             #Veremos algo do tipo: nome    AFHI11    Name: 0, dtype: object 
    pesquisa = pesquisa['nome']                 #Como só interessa a coluna nome, iremos filtrar por ela
    #print(pesquisa)                             #aqui teremos só a coluna nome do index, que será AFHI11
    url = (site_investimento+pesquisa)           #Puxando o Site_investimento (fiis.com),iremos criar a string com o nome
    #print(url)
    navegador.get(url)                           #Puxaremos então a url  fiis.com.br/nome atual
    #print('Novo CICLO!  ',ind)
    
    #Busca Elemento Na Tabela
    tabela = navegador.find_element(By.XPATH, '//*[@id="carbon_fields_fiis_dividends-2"]/div[2]/div[2]')
    #tentar procurar com a Classe 
    #tabela = navegador.find_element(By.CLASSNAME, 'dividends-table-container')
    texto = tabela.text
    texto = texto.strip().split('\n')
    
    df = pd.DataFrame([x.split(';') for x in texto if x])
    
    
    ###
    
    #nome_da_tabela = df[0][0]
    #print(nome_da_tabela)
    #print(df)
    
    #df_reshaped = pd.DataFrame(df.values.reshape(-1, 5), 
    #                           columns=['Data Base', 'Data Pagamento', 'Cotação Base', 'Dividend Yield', 'Rendimento'])

    ##df = pd.DataFrame({tabela.text})
    ####teste = teste.transpose
    ###print(teste)
    ###teste.to_csv(r'C:\Users\User\Downloads\teste.csv', sep='\n', encoding='ANSI',index=False)
    
    ##Busca Ultimo Rendimento Apenas
    #ult_rend = navegador.find_element(By.XPATH, '//*[@id="carbon_fields_fiis_header-2"]/div/div/div[1]/div[2]/div/div[2]/p[1]/b')
    #print(pesquisa + ' Rendeu no Ultimo mês ' + ult_rend.text)


In [ ]:
de = df #Dropa a primeira linha
i = 0
c = 0
coluna_df = pd.DataFrame()
result_df = coluna_df

for ind in de.values:
    if i==0:
        print('Dropar',ind)
    elif i > 0 and i < 6 :
        print('Cabeçalho', ind)
        #coluna_df = pd.DataFrame(columns=(ind))
        coluna_df[ind] = 'None'
        
        print(coluna_df)
    else:
        print(ind, i)
        print(c)
        # Atribua os valores pelo índice da coluna
        coluna_df[coluna_df.columns[c]] = ind
        c = c + 1
        if c >= 5:
            c = 0
        result_df = result_df.append(coluna_df, ignore_index=True)
    i=i+1
    
    
# Set the first column as the index
##de.set_index(0, inplace=False)
# Transpose the table
##de = de.transpose()

# Reset the index
##de.reset_index(inplace=True)
##de

#for ind in df.size:
#    print(ind)

#cabecalho = df[1:6].transpose(copy='True') #Transformamos as 5 primeiras linhas em colunas
#cab = cabecalho.values  #valores do cabecalho
#cabecalho
#itens = df[6:].transpose(copy='True') #Transformamos as 5 primeiras linhas em colunas
#itens

Dropar ['ÚLTIMOS DIVIDENDOS DO MALL11']
Cabeçalho ['Data Base']
Empty DataFrame
Columns: [Data Base]
Index: []
Cabeçalho ['Data Pagamento']
Empty DataFrame
Columns: [Data Base, Data Pagamento]
Index: []
Cabeçalho ['Cotação Base']
Empty DataFrame
Columns: [Data Base, Data Pagamento, Cotação Base]
Index: []
Cabeçalho ['Dividend Yield']
Empty DataFrame
Columns: [Data Base, Data Pagamento, Cotação Base, Dividend Yield]
Index: []
Cabeçalho ['Rendimento']
Empty DataFrame
Columns: [Data Base, Data Pagamento, Cotação Base, Dividend Yield, Rendimento]
Index: []
['29.09.2023'] 6
0
['16.10.2023'] 7
1
['R$ 117,69'] 8
2
['0,72%'] 9
3
['R$ 0,85'] 10
4
['31.08.2023'] 11
0
['15.09.2023'] 12
1
['R$ 117,31'] 13
2
['0,72%'] 14
3
['R$ 0,85'] 15
4
['31.07.2023'] 16
0
['14.08.2023'] 17
1
['R$ 115,50'] 18
2
['0,69%'] 19
3
['R$ 0,80'] 20
4
['30.06.2023'] 21
0
['14.07.2023'] 22
1
['R$ 117,48'] 23
2
['0,68%'] 24
3
['R$ 0,80'] 25
4
['31.05.2023'] 26
0
['15.06.2023'] 27
1
['R$ 109,50'] 28
2
['0,71%'] 29
3
['R$ 0,

C:\Users\User\AppData\Local\Temp\ipykernel_20032\1788396385.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(coluna_df, ignore_index=True)


In [ ]:
result_df.columns



Index(['Data Base', 'Data Pagamento', 'Cotação Base', 'Dividend Yield',
       'Rendimento'],
      dtype='object')

In [ ]:
result_df

   Data Base Data Pagamento Cotação Base Dividend Yield Rendimento
0          1            NaN          NaN            NaN        NaN
1        NaN              2          NaN            NaN        NaN
2        NaN            NaN            3            NaN        NaN
3        NaN            NaN          NaN              4        NaN
4        NaN            NaN          NaN            NaN          5
..       ...            ...          ...            ...        ...
95        96            NaN          NaN            NaN        NaN
96       NaN             97          NaN            NaN        NaN
97       NaN            NaN           98            NaN        NaN
98       NaN            NaN          NaN             99        NaN
99       NaN            NaN          NaN            NaN        100

[100 rows x 5 columns]
